In [1]:
import feedparser
import pandas as pd
import boto3
import json
from botocore.exceptions import ClientError
from langchain_core.prompts import PromptTemplate
import os
from dotenv import load_dotenv
from datetime import datetime
from bs4 import BeautifulSoup
import numpy as np
import time

In [2]:
RSS_list = [
    # --- グローバル・メガテック公式（最先端技術・基盤モデルの動向） ---
    "https://openai.com/news/rss.xml",           # OpenAI (業界のゲームチェンジャー、最新モデル発表)
    "https://cloudblog.withgoogle.com/rss/",     # Google Cloud Blog (Gemini/Vertex AI関連)
    "https://aws.amazon.com/jp/blogs/aws/feed/", # AWS News Blog (Bedrock/SageMaker関連)
    "https://developers-jp.googleblog.com/atom.xml", # Google Developers Japan (国内向け開発情報)

    # --- 国内メガベンチャー・テックブログ（実践的な実装・導入事例） ---
    "https://engineering.mercari.com/blog/feed.xml", # Mercari (大規模プロダクトへのAI実装)
    "https://developers.cyberagent.co.jp/blog/feed/", # CyberAgent (広告AI、LLM独自開発)
    "https://www.ai-shift.co.jp/techblog/feed",  # AI Shift (カスタマーサポートAI、RAG実践)
    "https://recruit-tech.co.jp/blog/feed/",     # Recruit (大規模データ活用・マッチングAI)
    "https://blog.brainpad.co.jp/rss",           # ブレインパッド (DXコンサル・データサイエンス)

    # --- 開発コミュニティ・技術メディア（現場のナレッジ・最新トレンド） ---
    "https://zenn.dev/topics/llm/feed",          # Zenn LLM (国内エンジニアの最速Tips・試行錯誤)
    "https://rss.itmedia.co.jp/rss/2.0/aiplus.xml", # ITmedia AI+ (最新ニュース・国内動向)
    "https://atmarkit.itmedia.co.jp/rss/rss.xml", # ＠IT (深掘り技術解説・マネジメント向け)
#    "https://xtech.nikkei.com/rss/xtech-it.rdf", # 日経クロステック (ビジネス視点のIT・AIニュース) #published_parsedがないかも
    "https://medium.com/feed/@kyakuno",          # Medium @kyakuno (技術選定・アーキテクチャ考察)

    # --- 論文・リサーチ（SOTA・学術的ブレイクスルー） ---
    "https://rsshub.app/huggingface/daily-papers", # Hugging Face Daily Papers (最重要論文)
    "https://research.preferred.jp/feed/",       # PFN Research (日本最高峰のAI研究)
    "https://feeds.feedburner.com/blogspot/gJZg", # Google Research Blog (グローバルな研究成果)
    "https://pytorch.org/feed.xml",              # PyTorch Official (深層学習フレームワークの進化)
#    "https://huggingface.co/blog/feed.xml",      # Hugging Face Blog (オープンソースモデルのハブ) #summaryがないかも

    # --- 海外ニュースレター・分析（戦略・マクロ・倫理） ---
    "https://www.deeplearning.ai/the-batch/rss", # The Batch (Andrew Ng氏監修の質の高い週刊分析)
    "https://www.technologyreview.jp/feed/",      # MIT Technology Review (技術の社会・倫理的インパクト)
]

In [ ]:
id_1 = 1
today = datetime.now().strftime("%Y%m%d")
category = "AI_news"
news_df = pd.DataFrame()

for url in RSS_list:
    feed = feedparser.parse(url)
    id_2 = 0
    id_3 = 1
    for entry in feed.entries[0:1]:
        # published_parsedをintオブジェクトに変換
        struct_time = entry.published_parsed
        dt = datetime(*struct_time[:6])
        published_datetime = dt.timestamp()
        published_datetime = int(published_datetime)

        # TTLの設定
        ttl = 14*24*60*60  # 14日間のTTLを秒単位で計算
        ttl = published_datetime + ttl

        print(f"ID: {today}{id_1}{id_2}{id_3}")
        print(f"カテゴリー: {category}")
        print(f"タイトル: {entry.title}")
        print(f"リンク: {entry.link}")
        print(f"公開日時 (datetime): {dt}")
        print(f"概要: {entry.summary}")
        print(f"TTL時間: {ttl}")

        addRow = [f"{today}{id_1}{id_2}{id_3}", category,  entry.title, entry.link, published_datetime, entry.summary, ttl]
        addRow = pd.DataFrame([addRow], columns=["id", "category", "title", "link", "published_datetime", "summary", "ttl"])
        news_df = pd.concat([news_df, addRow], ignore_index=True)
        print("-" * 30)

        if id_3 == 9:
            id_2 += 1
        
        if id_3 == 9:
            id_3 = 0
        else:
            id_3 += 1

    id_1 += 1

ID: 20260205101
カテゴリー: AI_news
タイトル: Navigating health questions with ChatGPT
リンク: https://openai.com/index/navigating-health-questions
公開日時 (datetime): 2026-02-05 00:00:00
概要: A family shares how ChatGPT helped them prepare for critical cancer treatment decisions for their son alongside expert guidance from his doctors.
TTL時間: 1770822000
------------------------------
ID: 20260205201
カテゴリー: AI_news
タイトル: The platform usage trap part 1: Why high activity doesn’t necessarily mean high value
リンク: https://cloud.google.com/blog/products/application-development/at-john-lewis-partnership-measuring-developer-platform-value/
公開日時 (datetime): 2026-02-04 18:00:00
概要: <div class="block-paragraph_advanced"><p><span style="font-style: italic; vertical-align: baseline;">For any organization that has invested in an internal developer platform, a question inevitably arises: Is it actually working? </span></p>
<p><span style="font-style: italic; vertical-align: baseline;">Simply tracking adoption rates

In [34]:
# HTMLタグを除去する関数
def clean_html(html, strip=False):
    soup = BeautifulSoup(html, 'html.parser')
    text = soup.get_text(strip=strip)
    return text
news_df["summary"] = news_df["summary"].apply(lambda x: clean_html(x,strip=True))

In [36]:
news_json = news_df.to_dict(orient='records')
table_name = 'test-project'
region_name = 'ap-northeast-1'

# dynamodbの設定
dynamodb = boto3.resource('dynamodb', region_name=region_name)
table = boto3.resource('dynamodb').Table(table_name)

# 一括書き込み。batch_writerを使用すると、DynamoDBの制限に基づいて自動的にバッチを分割してくれる。
with table.batch_writer() as batch:
    for item in news_json:
        item_not_has_nan = {key: item[key] for key in item if item[key] is not np.nan}
        batch.put_item(Item=item_not_has_nan)

In [ ]:
def load_api_key():
    load_dotenv(dotenv_path="./.env", override=True)
    os.environ["AWS_BEARER_TOKEN_BEDROCK"] = os.getenv("Bedrock_API_Key")

def create_response(prompt_text,df_json):
    client = boto3.client(
        "bedrock-runtime",
        region_name="us-east-1"
    )

    prompt = PromptTemplate(
        input_variables=["tweet_data"],
        template = prompt_text)
    prompt = prompt.format(tweet_data=df_json)

    body = json.dumps({
        "messages":[
            {"role":"user","content":prompt}
        ],
        "max_tokens":1024,
        "temperature":0.5,
        "anthropic_version":"bedrock-2023-05-31"
        
    })

    model_id = "anthropic.claude-3-5-sonnet-20240620-v1:0"

    response = client.invoke_model(
        modelId=model_id,
        body=body
    )

    return response

load_api_key()

f = open("prompt.txt","r",encoding="utf-8")
prompt_text = f.read()
response = create_response(prompt_text,df_json)

response_body = json.loads(response.get("body").read())
answer = response_body["content"][0]["text"]
print(answer)